In [ ]:
#Configuración del ambiente 
import pandas as pd
import numpy as np 
import requests
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
from sklearn.ensemble import RandomForestRegressor
import json

In [ ]:
df_covid = pd.read_csv(StringIO(requests.get("https://covid19.who.int/WHO-COVID-19-global-data.csv").text))
df_population = pd.read_excel('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT_REV1.xlsx', sheet_name=0, skiprows=16)
df_population = df_population[df_population['Year'] == 2019]

In [ ]:
#Aplicar .describe() .info() .boxplot()
df_covid.describe()

In [ ]:
# Hallar nulos
df_covid.isna().sum().sort_values(ascending = False)

In [ ]:
#Eliminar todos los registros con nulos en la columna Country_code.
df_covid = df_covid.dropna(subset=['Country_code'])
df_covid

In [ ]:
#Mantener en el dataframe sólo las siguientes columnas:
df_covid = df_covid[['Date_reported','Country_code','Country','New_cases','Cumulative_cases','New_deaths','Cumulative_deaths']]
df_covid

In [ ]:
#Sustituir todos los valores nulos con cero.
df_covid.loc[:, :] = df_covid.fillna(0)
df_covid

In [ ]:
#Cambiar el formato actual de la columna Date_reported para un formato de tipo datetime64[ns].
df_covid["Date_reported"] = pd.to_datetime(df_covid["Date_reported"])


In [ ]:
df_covid.info()

In [ ]:
#Substituir el formato actual de las columnas ['New_cases', 'New_deaths']para un formato de tipo int64.
df_covid.loc[:,"New_deaths"] = pd.to_numeric(df_covid["New_deaths"].astype('int64'))
df_covid.loc[:,"New_cases"] = pd.to_numeric(df_covid["New_cases"].astype('int64'))


In [ ]:
#Crear una nueva columna 'lethality_rate' con la siguiente fórmula: 'Cumulative_deaths'/'Cumulative_cases'*100
df_covid['lethality_rate'] = df_covid['Cumulative_deaths'] / df_covid['Cumulative_cases'] * 100
df_covid

In [ ]:
#Con el fin de eliminar outliers, nos quedaremos con los registros cuyo valor en la columna lethality_rate se encuentre entre los cuantiles 0 a 0.99.
df_covid = df_covid[(df_covid['lethality_rate'] > 0) & (df_covid['lethality_rate'] < 0.99)]
df_covid

In [ ]:
#Finalmente reiniciaremos el índice del dataframe final df_covid_limpio
df_covid_limpio = df_covid.reset_index(drop=True)
df_covid_limpio

In [ ]:
#Corroboramos cambios y que la limpieza ha sido efectuada
df_covid_limpio.info()

In [ ]:
#Almacenar el dataFrame en un csv para trabajar las etapas por separado.
df_covid_limpio.to_csv('df_covid_limpio.csv', sep=',', index=False)

In [ ]:
df_covid_limpio.info()

In [271]:
#Observamos el contenido del dataset
df_population

,Country_code,Total_Population,Male_Population,Female_Population,Population_Density,Life_Expectancy
1801,BI,11874000,5894000,5980000,457,62
1873,KM,790000,397000,393000,425,64
1945,DJ,1073000,533000,540000,46,63
2017,ER,3498000,1724000,1773000,28,67
2089,ET,114120000,57375000,56745000,114,65
...,...,...,...,...,...,...
20305,WS,211000,108000,103000,74,72
20377,TK,1000,0,0,146,75
20449,TO,104000,51000,52000,161,70
20521,TV,10000,5000,5000,365,64


In [272]:
#Mantener en el dataframe sólo las siguientes columnas: ['ISO2 Alpha-code','Total Population, as of 1 July (thousands)','Male Population, as of 1 July (thousands)','Female Population, as of 1 July (thousands)','Population Density, as of 1 July (persons per square km)','Life Expectancy at Birth, both sexes (years)'].
df_population = df_population[['ISO2 Alpha-code','Total Population, as of 1 July (thousands)','Male Population, as of 1 July (thousands)','Female Population, as of 1 July (thousands)','Population Density, as of 1 July (persons per square km)','Life Expectancy at Birth, both sexes (years)']]
df_population

KeyError: "None of [Index(['ISO2 Alpha-code', 'Total Population, as of 1 July (thousands)',\n       'Male Population, as of 1 July (thousands)',\n       'Female Population, as of 1 July (thousands)',\n       'Population Density, as of 1 July (persons per square km)',\n       'Life Expectancy at Birth, both sexes (years)'],\n      dtype='object')] are in the [columns]"

In [273]:
#Renombrar las columnas para nombre más sencillos:
df_population = df_population.rename(columns={
    'ISO2 Alpha-code': 'Country_code',
    'Total Population, as of 1 July (thousands)': 'Total_Population',
    'Male Population, as of 1 July (thousands)': 'Male_Population',
    'Female Population, as of 1 July (thousands)': 'Female_Population',
    'Population Density, as of 1 July (persons per square km)': 'Population_Density',
    'Life Expectancy at Birth, both sexes (years)': 'Life_Expectancy'
})
df_population

,Country_code,Total_Population,Male_Population,Female_Population,Population_Density,Life_Expectancy
1801,BI,11874000,5894000,5980000,457,62
1873,KM,790000,397000,393000,425,64
1945,DJ,1073000,533000,540000,46,63
2017,ER,3498000,1724000,1773000,28,67
2089,ET,114120000,57375000,56745000,114,65
...,...,...,...,...,...,...
20305,WS,211000,108000,103000,74,72
20377,TK,1000,0,0,146,75
20449,TO,104000,51000,52000,161,70
20521,TV,10000,5000,5000,365,64


In [274]:
#Hallar todos los registros nulos.
df_population.isna().sum().sort_values(ascending = False)

Country_code          0
Total_Population      0
Male_Population       0
Female_Population     0
Population_Density    0
Life_Expectancy       0
dtype: int64

In [275]:
# Borrar registros nulos
df_population = df_population.dropna()
df_population

,Country_code,Total_Population,Male_Population,Female_Population,Population_Density,Life_Expectancy
1801,BI,11874000,5894000,5980000,457,62
1873,KM,790000,397000,393000,425,64
1945,DJ,1073000,533000,540000,46,63
2017,ER,3498000,1724000,1773000,28,67
2089,ET,114120000,57375000,56745000,114,65
...,...,...,...,...,...,...
20305,WS,211000,108000,103000,74,72
20377,TK,1000,0,0,146,75
20449,TO,104000,51000,52000,161,70
20521,TV,10000,5000,5000,365,64


In [276]:
#Verificar el tipo de datos de las columnas
df_population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 1801 to 20593
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country_code        235 non-null    object
 1   Total_Population    235 non-null    object
 2   Male_Population     235 non-null    object
 3   Female_Population   235 non-null    object
 4   Population_Density  235 non-null    object
 5   Life_Expectancy     235 non-null    object
dtypes: object(6)
memory usage: 12.9+ KB


In [277]:
#Identificar valores único para encontrar el valor no numérico
print(df_population["Male_Population"].unique())


[5894000 397000 533000 1724000 57375000 25272000 13804000 9168000 640000
 139000 14851000 458000 6266000 54000 8011000 5167000 21241000 29563000
 9066000 7231000 15983000 12855000 2605000 8094000 2781000 44600000 821000
 1144000 107000 21756000 53429000 3329000 18281000 21607000 5962000 301000
 1233000 579000 1097000 28207000 6152000 10431000 286000 13218000 1247000
 15723000 6349000 971000 2479000 10387000 2145000 11889000 102680000 2000
 7860000 4030000 4142000 9005000 3105000 4696000 3050000 16495000
 726781000 3461000 312000 11802000 12707000 61158000 1606000 25894000
 19090000 82164000 407000 714325000 43792000 291000 13704000 113015000
 10440000 226000 8021000 135798000 3636000 16794000 26423000 56011000
 3066000 34723000 653000 47288000 1274000 5051000 933000 615000 1774000
 20788000 4287000 5555000 2738000 2814000 2924000 2055000 20836000 2446000
 10073000 41866000 6443000 15953000 4467000 3420000 5189000 4678000
 18625000 1478000 9450000 67697000 2663000 20480000 2881000 62700

In [ ]:
# Encontrar una fila con una condicion
n = df_population[df_population['Male_Population'] == '...']
n

In [ ]:
# Filtrar el DataFrame utilizando loc y realizar la eliminación
df_population.drop(df_population.loc[df_population['Male_Population'] == '...'].index, inplace=True)


In [282]:
#Substituir el formato actual de las columnas ['Total_Population','Male_Population','Female_Population','Population_Density','Life_Expectancy']para un formato de tipo int64.

df_population["Total_Population"] = pd.to_numeric(df_population["Total_Population"].astype('int64'))
df_population["Male_Population"] = pd.to_numeric(df_population["Male_Population"].astype('int64'))
df_population["Female_Population"] = pd.to_numeric(df_population["Female_Population"].astype('int64'))
df_population["Population_Density"] = pd.to_numeric(df_population["Population_Density"].astype('int64'))
df_population["Life_Expectancy"] = pd.to_numeric(df_population["Life_Expectancy"].astype('int64'))




In [283]:
df_population.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 1801 to 20593
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country_code        235 non-null    object
 1   Total_Population    235 non-null    int64 
 2   Male_Population     235 non-null    int64 
 3   Female_Population   235 non-null    int64 
 4   Population_Density  235 non-null    int64 
 5   Life_Expectancy     235 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 12.9+ KB


In [284]:
#Multiplicar por 1000 el valor actual de las siguientes columnas 'Total_Population','Male_Population','Female_Population' y sobreescribirlas.
df_population[['Total_Population','Male_Population','Female_Population']] = df_population[['Total_Population','Male_Population','Female_Population']] * 1000
df_population

,Country_code,Total_Population,Male_Population,Female_Population,Population_Density,Life_Expectancy
1801,BI,11874000000,5894000000,5980000000,457,62
1873,KM,790000000,397000000,393000000,425,64
1945,DJ,1073000000,533000000,540000000,46,63
2017,ER,3498000000,1724000000,1773000000,28,67
2089,ET,114120000000,57375000000,56745000000,114,65
...,...,...,...,...,...,...
20305,WS,211000000,108000000,103000000,74,72
20377,TK,1000000,0,0,146,75
20449,TO,104000000,51000000,52000000,161,70
20521,TV,10000000,5000000,5000000,365,64


In [285]:
# Buscar nulos
df_population.isna().sum().sort_values(ascending = False)

Country_code          0
Total_Population      0
Male_Population       0
Female_Population     0
Population_Density    0
Life_Expectancy       0
dtype: int64

In [286]:
#Finalmente reiniciaremos el índice del dataframe final df_population_limpio
# Reiniciar índice
df_population_limpio = df_population.reset_index(drop=True)
df_population_limpio

,Country_code,Total_Population,Male_Population,Female_Population,Population_Density,Life_Expectancy
0,BI,11874000000,5894000000,5980000000,457,62
1,KM,790000000,397000000,393000000,425,64
2,DJ,1073000000,533000000,540000000,46,63
3,ER,3498000000,1724000000,1773000000,28,67
4,ET,114120000000,57375000000,56745000000,114,65
...,...,...,...,...,...,...
230,WS,211000000,108000000,103000000,74,72
231,TK,1000000,0,0,146,75
232,TO,104000000,51000000,52000000,161,70
233,TV,10000000,5000000,5000000,365,64


In [287]:
df_population_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country_code        235 non-null    object
 1   Total_Population    235 non-null    int64 
 2   Male_Population     235 non-null    int64 
 3   Female_Population   235 non-null    int64 
 4   Population_Density  235 non-null    int64 
 5   Life_Expectancy     235 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 11.1+ KB


In [288]:
#Almacenar el dataFrame en un csv para trabajar las etapas por separado.
df_population_limpio.to_csv('df_population_limpio.csv', sep=',', index=False)

In [289]:
df_population_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Country_code        235 non-null    object
 1   Total_Population    235 non-null    int64 
 2   Male_Population     235 non-null    int64 
 3   Female_Population   235 non-null    int64 
 4   Population_Density  235 non-null    int64 
 5   Life_Expectancy     235 non-null    int64 
dtypes: int64(5), object(1)
memory usage: 11.1+ KB
